In [5]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # choose to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc


In [6]:
def ctrl_increment(qc, x, ctrl):
    """ This function performs a controlled increment with 
        NO overflow handling
    
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - x: QuantumRegister for the number to be incremented
        - ctrl: the control qubit for performing the operation

        Returns:
        - QuantumCircuit: The updated quantum circuit where x has been incremented 
            conditioned on the ctrl
    """
    n=len(x)

    ''' Ancilla adder '''
    # Dynamically add carry qubits
    c = QuantumRegister(n - 1, 'carry')
    qc.add_register(c)

    qc.ccx(ctrl[0], x[0], c[0])

    #apply TempAND computes
    for i in range(1,n-1):
        qc.ccx(c[i-1], x[i], c[i])

    #end segment
    qc.cx(c[n-2],x[n-1])

    # TempAND un-computes and carry increments
    for i in range(n-2,0,-1): # i goes from n-2 to 1
        qc.ccx(c[i-1], x[i], c[i])
        qc.cx(c[i-1], x[i])

    qc.ccx(ctrl[0], x[0], c[0])
    qc.cx(ctrl[0], x[0])
    
    ''' Ancilla removal '''
    # Remove carry qubits to restore the original circuit size
    for i in range(0,n-1):
        qc.qubits.pop()
    
    return qc

In [7]:
""" MAIN """

# Number of bits
n = 5

ctrl = QuantumRegister(1, 'ctrl')
x = QuantumRegister(n, 'x')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(n, 'result')

qc = QuantumCircuit(x, ctrl, result)

"""initialize the quantum numbers"""
#qc = init_random_num(qc, x)
for i in range(0,n-1):
    qc.x(x[i])

""" Initialize the ctrl """
#qc.x(ctrl[0]) #control

#apply the ctrl add/sub operation
qc = ctrl_increment(qc, x, ctrl)

# Measure the sum qubits and carry qubit
#qc.measure_all()

# Measure the qubits into the classical register
qc.measure(x, result[:n])      # Measure 'x' into the first n classical bits


# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

print("Measurement results:")
for bitstring, count in counts.items():
    # bitstring is of the form 'result'
    x_result = bitstring  # First n bits for x
    x_decimal_value = int(x_result, 2)
        
    print(f"x = {x_result} (decimal: {x_decimal_value})")

{'01111': 1024}
Measurement results:
x = 01111 (decimal: 15)
